In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from rcrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_rcrp, get_docs
from configure import get_config

# load config & data 

In [5]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [6]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [7]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

In [8]:
docs_bow = [instance.bow for instance in instances_train]
docs_raw = [[[bow_index]*int(doc_bow[bow_index]) for bow_index in np.where(doc_bow > 0)[0]] for doc_bow in docs_bow]
docs_words = [[idx for idxs in doc for idx in idxs] for doc in docs_raw]
np.sum([len(doc_words) for doc_words in docs_words])

568401

# run

## initialize log

In [9]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [10]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=1, config=config)
train_docs = get_docs(instances_train, config, topic_root=topic_root)
dev_docs = get_docs(instances_dev, config, topic_root=topic_root)
test_docs = get_docs(instances_test, config, topic_root=topic_root)
init(train_docs, dev_docs, test_docs, topic_root)

## run

In [11]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train = 0.
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2] if 2 in depth_spec else 0, '%.2f'%depth_spec[3] if 3 in depth_spec else 0, \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_rcrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   192   0  0      0    524   517  0.05  0.47  0.54  0.52  0.69
1   220   1  0      0    509   498  0.05  0.49  0.54  0.51  0.63
2   233   2  0      0    489   472  0.05  0.50  0.53  0.44  0.67
3   243   3  0      0    473   453  0.05  0.49  0.54  0.39  0.70
4   237   4  0      0    461   441  0.05  0.47  0.53  0.60  0.57
5   234   5  0      0    453   431  0.05  0.46  0.53  0.54  0.54
6   248   6  0      0    444   421  0.05  0.47  0.53  0.61  0.60
7   266   7  0      0    436   418  0.05  0.46  0.53  0.58  0.58
8   277   8  0      0    430   413  0.05  0.47  0.53  0.55  0.52
9   282   9  0      0    422   406  0.05  0.47  0.53  0.52  0.49
10  278  10  0      0    415   399  0.05  0.45  0.53  0.51  0.48
11  289  11  0      0    405   390  0.05  0.44  0.53  0.46  0.44
12  290  12  0      0    395   380  0.05  0.45  0.55  0.40  0.47
13  305  13  0      0    387   373  0.05  0.44  0.53  0.33  0.51
14  305  14  0      0    377   368  0.05  0.43  0.53  0.37  0.49
15  307  15  0      0    372   363  0.05  0.43     0  0.00  0.00
16  307  16  0      0    366   357  0.05  0.42  0.53  0.36  0.47
17  333  17  0      0    365   354  0.05  0.42  0.52  0.42  0.46
18  344  18  0      0    359   348  0.05  0.41  0.53  0.36  0.48
19  321  19  0      0    354   346  0.05  0.41     0  0.00  0.00
20  324  20  0      0    354   343  0.05  0.42     0  0.00  0.00
21  338  21  0      0    347   339  0.05  0.41  0.53  0.35  0.54
22  343  22  0      0    347   339  0.05  0.41  0.52  0.51  0.52
23  375  23  0      0    346   336  0.05  0.41  0.52  0.65  0.51
24  356  24  0      0    347   336  0.05  0.41     0  0.00  0.00
25  347  25  0      0    341   334  0.05  0.41  0.52  0.50  0.51
26  360  26  0      0    339   331  0.05  0.42  0.53  0.46  0.56
27  369  27  0      0    337   329  0.05  0.42  0.52  0.50  0.56
28  368  28  0      0    335   330  0.05  0.42     0  0.00  0.00
29  361  29  0      0    334   328  0.05  0.41     0  0.00  0.00

   0 : ['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8'] 666 3754.0 ['!', 'love', 'cover', 'nice', 'sleeve', ';', 'carry', '&', 'bought', 'price']
     0-1 : [] 23699 135306.0 ['!', 'cover', 'color', 'love', 'bottom', 'keyboard', 'mac', 'pro', 'nice', 'hard']
     0-2 : [] 21794 127642.0 ['carry', 'back', '!', 'strap', 'quality', 'handle', 'bought', 'work', 'pockets', 'straps']
     0-3 : [] 20634 125270.0 ['pocket', 'carry', 'room', 'pockets', '!', 'small', 'perfect', 'hold', 'ipad', 'inside']
     0-4 : [] 17808 95750.0 ['sleeve', '&', ';', 'protection', 'inside', 'pro', 'inch', 'nice', 'air', 'zipper']
     0-5 : [] 3297 19735.0 ['netbook', 'drive', 'cards', 'usb', 'power', 'memory', 'hard', 'dvd', 'small', 'card']
     0-6 : [] 11198 57482.0 ['!', 'item', '...', 'price', 'amazon', 'quality', 'smell', 'received', 'return', 'ordered']
     0-7 : [] 498 3291.0 ['water', 'open', 'version', 'lap', 'flap', '`', "'", 'customer', 'top', 'design']
     0-8 : [] 21 171.0 ['stand', 'co